In [5]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from sklearn.decomposition import NMF

# 1. Chargement des données propres
print("Chargement des données...")
X = load_npz('../out/user_permission_matrix_sparse.npz')
users_df = pd.read_csv('../out/users_catalog.csv')
perms_df = pd.read_csv('../out/perm_catalog.csv')
apps_df = pd.read_csv('../out/app_catalog.csv')

# 2. Configuration et Entraînement de la NMF
N_ROLES = 15  # On cherche 15 rôles latents (comme pour le K-Means)

print(f"Entraînement de la NMF avec {N_ROLES} composants...")
# init='nndsvd' est une méthode d'initialisation optimisée pour les matrices creuses
model = NMF(n_components=N_ROLES, init='nndsvd', random_state=42, max_iter=500, solver='mu')

# W : Matrice Utilisateurs x Rôles (Poids de chaque utilisateur pour chaque rôle)
W = model.fit_transform(X)

# H : Matrice Rôles x Permissions (Composition de chaque rôle)
H = model.components_

print("Entraînement terminé !")
print(f"Dimensions W (Users x Roles) : {W.shape}")
print(f"Dimensions H (Roles x Perms) : {H.shape}")

Chargement des données...
Entraînement de la NMF avec 15 composants...
Entraînement terminé !
Dimensions W (Users x Roles) : (5000, 15)
Dimensions H (Roles x Perms) : (15, 324)


/Users/tidianetall/Desktop/Desktop/Cours/PCE/Wavestone-52/venv/lib/python3.13/site-packages/sklearn/decomposition/_nmf.py:1581: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(


In [6]:
def afficher_top_permissions_role(role_id, n_top=8):
    # On récupère la ligne correspondant au rôle dans H
    role_weights = H[role_id]
    
    # On trie les indices des permissions par poids décroissant
    top_indices = role_weights.argsort()[::-1][:n_top]
    
    # On récupère les infos
    top_perms = perms_df.iloc[top_indices].copy()
    top_perms['weight'] = role_weights[top_indices] # On ajoute le poids NMF
    
    # Merge avec les noms d'apps pour lisibilité
    res = pd.merge(top_perms, apps_df, on='application_id', how='left')
    
    print(f"\n=== RÔLE {role_id} (Composition) ===")
    print(res[['perm_name', 'app_name', 'weight']].to_string(index=False))

# Affichons les 5 premiers rôles pour voir
for i in range(5):
    afficher_top_permissions_role(i)


=== RÔLE 0 (Composition) ===
        perm_name           app_name   weight
  manage_pipeline  GitHub Enterprise 3.749209
            login CrowdStrike Falcon 3.738219
      open_ticket    ServiceNow ITSM 3.732084
    open_employee         Talentsoft 3.728836
manage_identities CrowdStrike Falcon 3.724037
            login  GitHub Enterprise 3.722245
  manage_pipeline         Confluence 3.721885
      manage_team               Zoom 0.294405

=== RÔLE 1 (Composition) ===
        perm_name                app_name   weight
            login             Workday HCM 3.239307
    edit_employee              Talentsoft 3.227874
            login               Sage Paie 3.227061
  approve_payment Oracle Financials Cloud 3.217469
    edit_employee             Workday HCM 3.211435
export_hr_reports              Talentsoft 3.199561
    open_employee              Talentsoft 3.196602
     post_message         Exchange Online 0.229266

=== RÔLE 2 (Composition) ===
      perm_name                app_na

In [7]:
# Fonction pour voir les rôles d'un utilisateur spécifique
def afficher_roles_utilisateur(user_idx, threshold=0.1):
    # On récupère les infos de l'user
    user_info = users_df.iloc[user_idx]
    print(f"\n--- Analyse User {user_info['user_id']} ({user_info['position']} - {user_info['department']}) ---")
    
    # On récupère ses poids dans W
    user_weights = W[user_idx]
    
    # On affiche les rôles où il a un poids significatif (> threshold)
    # enumerate permet d'avoir (index_role, poids)
    roles_significatifs = [(i, w) for i, w in enumerate(user_weights) if w > threshold]
    
    # Tri par importance
    roles_significatifs.sort(key=lambda x: x[1], reverse=True)
    
    if not roles_significatifs:
        print("Aucun rôle dominant détecté (poids faibles partout).")
    else:
        for r_id, weight in roles_significatifs:
            print(f"  -> Rôle {r_id} : Score {weight:.4f}")

# Testons sur quelques utilisateurs au hasard
import random
random.seed(42)
sample_indices = random.sample(range(len(users_df)), 5)

for idx in sample_indices:
    afficher_roles_utilisateur(idx)


--- Analyse User 913 (Product Owner - Department Product) ---
  -> Rôle 4 : Score 0.2695

--- Analyse User 205 (Security Engineer - Department IT) ---
  -> Rôle 0 : Score 0.2691

--- Analyse User 2254 (Receptionist - Department Facilities) ---
Aucun rôle dominant détecté (poids faibles partout).

--- Analyse User 2007 (Scientist - Department R&D) ---
  -> Rôle 3 : Score 0.2611

--- Analyse User 1829 (Accountant - Department Finance) ---
  -> Rôle 2 : Score 0.3286


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import joblib
import os

print("--- PRÉPARATION DU CLASSIFIEUR SUPERVISÉ ---")

# 1. PRÉPARER LES DONNÉES (X et Y)
# --------------------------------
# Y (La Cible) : Quel est le rôle dominant pour chaque utilisateur existant ?
# On prend l'index du rôle avec le poids le plus fort dans la matrice W
dominant_roles = W.argmax(axis=1)
users_df['target_role_id'] = dominant_roles

# X (Les Features) : Les infos RH (Département, Position)
# On encode le texte en nombres pour que le modèle comprenne
le_dept = LabelEncoder()
users_df['dept_encoded'] = le_dept.fit_transform(users_df['department'])

le_pos = LabelEncoder()
users_df['pos_encoded'] = le_pos.fit_transform(users_df['position'])

# Si une colonne 'location' existe, on l'utilise, sinon on s'en passe
use_location = 'location' in users_df.columns
if use_location:
    le_loc = LabelEncoder()
    users_df['loc_encoded'] = le_loc.fit_transform(users_df['location'])

# Notre matrice de features X
if use_location:
    X_features = users_df[['dept_encoded', 'pos_encoded', 'loc_encoded']]
else:
    X_features = users_df[['dept_encoded', 'pos_encoded']]

y_target = users_df['target_role_id']

# 2. ENTRAINEMENT (Train/Test Split)
# ----------------------------------
# On garde 20% des données pour tester si le modèle est bon
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=42)

# Création du Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 3. ÉVALUATION
# -------------
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Précision du modèle : {acc*100:.2f}%")
print("\n--- Rapport détaillé ---")
# On affiche la précision pour chaque rôle
print(classification_report(y_test, y_pred))

# 4. SIMULATION : UN NOUVEL ARRIVANT ! 🆕
# ---------------------------------------
print("\n--- TEST AVEC UN NOUVEL ARRIVANT ---")

# Imaginons un nouveau "Recruiter" qui arrive au service "Department HR" à "Paris"
new_user_dept = "Department HR"
new_user_pos = "Recruiter"
new_user_loc = "Paris"

print(f"Nouvel utilisateur : {new_user_pos} chez {new_user_dept} ({new_user_loc})")

# On doit encoder ses infos comme pour l'entraînement
try:
    encoded_dept = le_dept.transform([new_user_dept])[0]
    encoded_pos = le_pos.transform([new_user_pos])[0]
    features = [encoded_dept, encoded_pos]
    if use_location:
        encoded_loc = le_loc.transform([new_user_loc])[0]
        features.append(encoded_loc)

    # Prédiction
    predicted_role_id = clf.predict([features])[0]

    print(f"🤖 Le modèle suggère d'attribuer le : Rôle {predicted_role_id}")

    # Affichons ce que contient ce rôle pour vérifier
    afficher_top_permissions_role(predicted_role_id)

except Exception as e:
    print(f"Erreur pendant la simulation du nouvel arrivant : {e}")

# 5. SAUVEGARDE DU MODÈLE (Pour l'utiliser ailleurs)
# --------------------------------------------------
# On sauvegarde le cerveau (clf) et les traducteurs (le_*)
out_dir = '../out'
os.makedirs(out_dir, exist_ok=True)
joblib.dump(clf, f'{out_dir}/role_predictor_model.pkl')
joblib.dump(le_dept, f'{out_dir}/encoder_dept.pkl')
joblib.dump(le_pos, f'{out_dir}/encoder_pos.pkl')
if use_location:
    joblib.dump(le_loc, f'{out_dir}/encoder_loc.pkl')
print(f"\nModèle sauvegardé dans {out_dir} !")

--- PRÉPARATION DU CLASSIFIEUR SUPERVISÉ ---
Précision du modèle : 87.00%

--- Rapport détaillé ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       261
           1       1.00      1.00      1.00        97
           2       1.00      1.00      1.00       109
           3       0.97      0.99      0.98        95
           4       0.94      0.93      0.93       100
           5       0.93      0.86      0.89       137
           6       0.47      1.00      0.64        98
           7       0.00      0.00      0.00        18
           8       0.00      0.00      0.00        13
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00         9
          11       0.00      0.00      0.00        17
          12       0.00      0.00      0.00         7
          13       0.00      0.00      0.00        13
          14       0.00      0.00      0.00        17

    accuracy                      

/Users/tidianetall/Desktop/Desktop/Cours/PCE/Wavestone-52/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/tidianetall/Desktop/Desktop/Cours/PCE/Wavestone-52/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/tidianetall/Desktop/Desktop/Cours/PCE/Wavestone-52/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi